In [2]:
import numpy as np
import pandas as pd
import requests
from selenium.webdriver.chrome.options import Options
import datetime as dt
from bs4 import BeautifulSoup
import json
import time
import numpy as np
from matplotlib import pyplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import re
import nltk
# from gensim.models import KeyedVectors
import pickle
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import re                                  # library for regular expression operations
import string                              # for string operations
from sklearn.decomposition import PCA
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings
from sklearn.preprocessing import StandardScaler
import os

In [3]:
def plot_chart(stock):
    fig = go.Figure()
    # Add candlestick data
    fig.add_trace(go.Candlestick(
        x=stock['Datetime'],
        open=stock['Open'],
        high=stock['High'],
        low=stock['Low'],
        close=stock['Close']
    ))

    fig.show()



In [64]:
def get_stock_data(ticker):
    stock=pd.read_csv(f'C:\\Users\\mahdi\\Desktop\\stock data\\Data2023_1D\\{ticker}.csv')
    stock.drop(['Unnamed: 0'],axis=1,inplace=True)
    return stock


def cluster_zones(zones,candle_volume):
    new_zones=[]
    i=0
    while i<len(zones)-1:
        j=i+1
        new_zone=0
        while j<len(zones) and abs((zones[i]-zones[j]))/candle_volume<=1.5:
            new_zone=(zones[i]+zones[j])/2
            j+=1
        if new_zone>0:
            new_zones.append(round(new_zone,2))
        else:
            new_zones.append(zones[i])
        i=j
    return new_zones
def get_zones(stock):
    zones=[]
    for i in range(1,len(stock)-2):
        bullish0=stock['Close'].iloc[i-1]>stock['Open'].iloc[i-1]
        bullish1=stock['Close'].iloc[i]>stock['Open'].iloc[i]
        bullish2=stock['Close'].iloc[i+1]>stock['Open'].iloc[i+1]
        if bullish0 and bullish1 and bullish2==False:
            zones.append(stock['High'].iloc[i])
        if bullish0==False and bullish1 and bullish2:
            zones.append(stock['Low'].iloc[i])
    zones=np.sort(zones)
    return zones

def candle_construction(df,candle_volume,zones):
    df.loc[:, "price_progress"] = (df['Close'] - df['Close'].shift(1)) / candle_volume
    diff_zones=[(df['Close'].iloc[len(df)-1]-zones[i])/candle_volume for i in range(len(zones))]
    df=df[['price_progress']]
    diff_zones=np.sort(diff_zones)
    neg_diffzone=-1000
    pos_diffzone=1000
    for i in range(len(diff_zones)):
        if diff_zones[i]<0:
            neg_diffzone=diff_zones[i]
        else:
            pos_diffzone=diff_zones[i]
            break
    if abs(neg_diffzone)<pos_diffzone:
        diff_zones=neg_diffzone
    else:
        diff_zones=pos_diffzone
#     x=df[1:len(df)-1].values
    return (df[1:],diff_zones)




def get_file_name_if_size_greater_than(file_path, size_threshold_kb):
    try:
        # Get the size of the file in bytes
        size_in_bytes = os.path.getsize(file_path)

        # Convert the size to kilobytes
        size_in_kb = size_in_bytes / 1024.0

        # Check if the file size is greater than the threshold
        if size_in_kb > size_threshold_kb:
            file_name_without_extension = os.path.splitext(os.path.basename(file_path))[0]
            return file_name_without_extension

    except FileNotFoundError:
        return None    
    
    
def generate_planes(dim=10):
    first_plan=np.random.random(dim)
    second_plan=np.random.random(dim)
    third_plan=np.random.random(dim)
    return first_plan,second_plan,third_plan

first_plane0,second_plane0,third_plane0=generate_planes()
first_plane1,second_plane1,third_plane1=generate_planes()
first_plane2,second_plane2,third_plane2=generate_planes()

def get_hashcode(x,first_plane,second_plane,third_plane):
    hash_code=[0,0,0]
    if np.dot(x,first_plane)>=0:
        hash_code[0]=1
    if np.dot(x,second_plane)>=0:
        hash_code[1]=1
    if np.dot(x,third_plane)>=0:
        hash_code[2]=1
    return hash_code

In [ ]:
folder_path = 'C:\\Users\\mahdi\\Desktop\\stock data\\Data2023_1D\\'
xlsx_files = glob.glob(folder_path + '*.csv')
tickers=[]
index_to_delete=[]
lookback=10
for file in xlsx_files:
    size_threshold_kb = 0
    tickers.append(get_file_name_if_size_greater_than(file, size_threshold_kb))
for ticker in tickers:
    print(ticker)
    stock=get_stock_data(ticker)
    zones=get_zones(stock)
    final_data=pd.DataFrame(columns=([f'pp_{j}' for j in range(1,lookback+1)]))
    for i in range(0,len(stock)):
        df=stock[i:i+lookback]
        candle_volume=(stock['High']-stock['Low']).median()
        zones1=cluster_zones(zones.copy(),candle_volume)
        df,diff_zones=candle_construction(df.copy(),candle_volume,zones1)
#         df.to_csv('')
        x=np.append(df.values,diff_zones)
        if x.shape[0]==lookback:
            hash_code0=get_hashcode(x,first_plane0,second_plane0,third_plane0)
            hash_code1=get_hashcode(x,first_plane1,second_plane1,third_plane1)
            hash_code2=get_hashcode(x,first_plane2,second_plane2,third_plane2)
            x=x.reshape(1,-1)
            df=pd.DataFrame(x,columns=([f'pp_{j}' for j in range(1,lookback+1)]))
            df['start_date']=stock['Datetime'].iloc[i]
            df['hash0_0']=hash_code0[0]
            df['hash0_1']=hash_code0[1]
            df['hash0_2']=hash_code0[2]
            
            df['hash1_0']=hash_code1[0]
            df['hash1_1']=hash_code1[1]
            df['hash1_2']=hash_code1[2]
            df['hash2_0']=hash_code2[0]
            df['hash2_1']=hash_code2[1]
            df['hash2_2']=hash_code2[2]
            final_data=pd.concat([final_data,df])

    final_data['ticker']=ticker
    final_data.to_csv(f'C:\\Users\\mahdi\\Desktop\\workshops\\Python workshop\\tradeProject\\models\\SameHistory detection\\data\\{ticker}.csv',index=False)
        

In [68]:
np.save('first_plane0.npy', first_plane0)
np.save('second_plane0.npy', second_plane0)
np.save('third_plane0.npy', third_plane0)

np.save('first_plane1.npy', first_plane1)
np.save('second_plane1.npy', second_plane1)
np.save('third_plane1.npy', third_plane1)

np.save('first_plane2.npy', first_plane2)
np.save('second_plane2.npy', second_plane2)
np.save('third_plane2.npy', third_plane2)